In [ ]:
import os
import openai
import pinecone
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time
from json import JSONDecodeError
import concurrent.futures
import traceback
import requests
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from PIL import Image
import pytesseract
from io import BytesIO

#INSERT API KEYS HERE
pinecone.init(api_key="", environment="us-west4-gcp")
index = pinecone.Index(index_name='gpt-dashboard-db')
os.environ['OPENAI_API_KEY'] = ""
openai.api_key = ""
embed_id = 1


In [ ]:
import logging
import os
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import json
import re
from datetime import datetime, timedelta
import time

#INSERT API KEY HERE
client = WebClient(token="") 
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

#Enter the channel names and their IDs here. E.g.
#   "random-core": "XYZ",
#   "quotes": "XYZ",
channels = {}

#this get user names and replace user ids with names
with open('user_names.json', 'r') as f:
    user_names = json.load(f)

def replace_user_ids_with_names(message):
    user_ids = re.findall('<@(\w+)>', message)
    for user_id in user_ids:
        user_name = user_names.get(user_id, 'Unknown User')
        message = message.replace(f'<@{user_id}>', f'@{user_name}')
    return message

def get_all_messages_with_threads_from_channel(workspace_name):
    for channel_name, channel_id in channels.items():
        all_threads = []
        try:
            client.conversations_join(channel=channel_id)  # replace bot_id with your bot's user id
            logger.info(f"Unarchiving channel {channel_name}")
            client.conversations_unarchive(channel=channel_id)
            logger.info(f"Adding bot to channel {channel_name}")
            response = client.conversations_history(channel=channel_id,
                                                    inclusive=True,
                                                    limit=950)
        except SlackApiError as e:
            if e.response["error"] == "ratelimited":
                delay = int(e.response.headers.get('Retry-After'))
                logger.info(f"Rate limit exceeded. Retrying in {delay} seconds")
                time.sleep(delay)
                logger.info(f"SKIPPED {channel_name}")
                continue
            else:
                raise e

        if response["ok"]:
            while True:
                messages = response["messages"]
                for message in messages:
                    time.sleep(0.45) 
                    thread_ts = message.get('thread_ts')
                    if thread_ts:
                        try:
                            thread_response = client.conversations_replies(channel=channel_id,
                                                                          ts=thread_ts,
                                                                          limit=999)
                        except SlackApiError as e:
                            if e.response["error"] == "ratelimited":
                                delay = int(e.response.headers.get('Retry-After'))
                                logger.info(f"Rate limit exceeded. Retrying in 60 seconds")
                                time.sleep(delay)
                                logger.info(f"SKIPPED {channel_name}")
                                continue
                            else:
                                raise e

                        if thread_response["ok"]:
                            thread_messages = thread_response['messages']
                            thread_data = []
                            for thread_message in thread_messages:
                                thread_user_id = thread_message.get('user')
                                if thread_user_id:
                                    thread_user_name = user_names.get(thread_user_id, 'Unknown User')

                                timestamp = datetime.fromtimestamp(float(thread_message['ts']))
                                thread_data.append({
                                    "name": thread_user_name,
                                    "message": replace_user_ids_with_names(thread_message['text']),
                                    "time": timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                                    "timestamp": timestamp.timestamp(),
                                })

                            thread_url = f"https://{workspace_name}.slack.com/archives/{channel_id}/p{thread_ts.replace('.', '')}"
                            all_threads.append({
                                'thread_url': thread_url,
                                'thread_data': thread_data
                            })

                if response['has_more']:
                    try:
                        response = client.conversations_history(
                            channel=channel_id,
                            inclusive=True,
                            limit=950,
                            cursor=response['response_metadata']['next_cursor'])
                    except SlackApiError as e:
                        if e.response["error"] == "ratelimited":
                            delay = int(e.response.headers.get('Retry-After'))
                            logger.info(f"Rate limit exceeded. Retrying in 60 seconds")
                            time.sleep(delay)
                            continue
                        else:
                            raise e
                else:
                    break

        with open(f'{channel_name}.json', 'w', encoding='utf8') as f:
            json.dump(all_threads, f, ensure_ascii=False)
        logger.info(f"{channel_name} done")

        try:
            logger.info(f"Archiving channel {channel_name}")
            client.conversations_archive(channel=channel_id)
        except SlackApiError as e:
            logger.error(f"Failed to archive channel {channel_name}: {e.response['error']}")

#INSERT YOUR SLACK WORKSPACE NAME HERE
get_all_messages_with_threads_from_channel("workspace")


In [ ]:
#unarhive and archive a channel (doesn't work currently)
import slack_sdk
import time

# Create a client instance 
client = WebClient(token="") #<----INSERT API KEY HERE

# ID of the channel to unarchive and archive

channel_id = '' #<----INSERT CHANNEL ID HERE

# Unarchive the channel
unarchive_response = client.conversations_unarchive(
    channel=channel_id
)

# Check if the channel was successfully unarchived
if unarchive_response['ok']:
    print(f'Channel {channel_id} was successfully unarchived')

    # Give some time before archiving again
    time.sleep(5)

    # Archive the channel
    archive_response = client.conversations_archive(
        channel=channel_id
    )

    # Check if the channel was successfully archived
    if archive_response['ok']:
        print(f'Channel {channel_id} was successfully archived')
    else:
        print(f'Failed to archive channel {channel_id}')
else:
    print(f'Failed to unarchive channel {channel_id}')
